# Relative Free Energy Alchemical Network Planners

To simplify the process of creating an [Alchemical Network] for Relative Free Energy calculations, openfe offers two convenience `Alchemical Network Planner` classes; [RBFEAlchemicalNetworkPlanner] and [RHFEAlchemicalNetworkPlanner].

[Alchemical Network]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.AlchemicalNetwork.html
[RBFEAlchemicalNetworkPlanner]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.setup.RBFEAlchemicalNetworkPlanner.html#openfe.setup.RBFEAlchemicalNetworkPlanner
[RHFEAlchemicalNetworkPlanner]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.setup.RHFEAlchemicalNetworkPlanner.html#openfe.setup.RHFEAlchemicalNetworkPlanner 

## Overview

To create an `AlchemicalNetwork` using the Alchemical Planners, the following steps must be followed:

1. Create appropriate `ChemicalComponents`
2. Choosing an appropriate atom mapper, scorer, and network generator
3. Create an `Alchemical Planner`
4. Create the `AlchemicalNetwork`

## Relative Binding Free Energy (RBFE)

Here we demonstrate how to use the [RBFEAlchemicalNetworkPlanner] to create an RBFE [AlchemicalNetwork]

[AlchemicalNetwork]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.AlchemicalNetwork.html
[RBFEAlchemicalNetworkPlanner]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.setup.RBFEAlchemicalNetworkPlanner.html#openfe.setup.RBFEAlchemicalNetworkPlanner

### Creating `ChemicalComponents`

First we define the `ChemicalComponents` which we want to represent in our `AlchemicalNetwork`.

In [1]:
from rdkit import Chem
from openfe import SmallMoleculeComponent, SolventComponent, ProteinComponent

In [2]:
# Some SmallMoleculeComponents defining the ligands we want to transform between
ligands = [
    SmallMoleculeComponent(m)
    for m in Chem.SDMolSupplier("assets/somebenzenes.sdf", removeHs=False)
]

# The ProteinComponent which will be present in our complex simulations
protein = ProteinComponent.from_pdb_file("assets/t4_lysozyme.pdb")

# The SolventComponent defining our system's solvation
solvent = SolventComponent()

### Choosing an atom mapper, scorer and network generator

Here we choose to use the [KartografAtomMapper] alongside the [Lomap scorer].

For our network we choose to create a minimum spanning network using [generate_minimal_spanning_network].

[KartografAtomMapper]: https://kartograf.readthedocs.io/en/latest/api/kartograf.mappers.html
[Lomap scorer]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.setup.atom_mapping.lomap_scorers.default_lomap_score.html
[generate_minimal_spanning_network]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.setup.ligand_network_planning.generate_minimal_spanning_network.html

In [3]:
from openfe.setup.atom_mapping import KartografAtomMapper
from openfe.setup.atom_mapping.lomap_scorers import default_lomap_score
from openfe.setup.ligand_network_planning import generate_minimal_spanning_network

### Building the Network Planner and getting an AlchemicalNetwork

Finally we can use the `RBFEAlchemicalNetworkPlanner` to create an `AlchemicalNetwork`.

Internally the Network Planner will:

1. Create a `LigandNetwork` (accessible under the `._ligand_network` attribute).
2. Create two `Transformation` per edge of the `LigandNetwork`, one for each of the complex and solvent legs of the RBFE alchemical cycle.
3. Add all `Transformation`s to a single `AlchemicalNetwork`.

In [4]:
from openfe.setup.alchemical_network_planner import RBFEAlchemicalNetworkPlanner

# Build the planner
alchem_planner = RBFEAlchemicalNetworkPlanner(
    name="project", 
    mappers=[KartografAtomMapper()],
    mapping_scorer=default_lomap_score,
    ligand_network_planner=generate_minimal_spanning_network,
)

# Create the AlchemicalNetwork
alchemical_network = alchem_planner(
    ligands=ligands,
    solvent=solvent,
    protein=protein
)

## Relative Hydration Free Energy (RHFE)

Similarly, an RHFE `AlchemicalNetwork` can be created using the [RHFEAlchemicalNetworkPlanner].

[RHFEAlchemicalNetworkPlanner]: https://docs.openfree.energy/en/stable/reference/api/generated/openfe.setup.RHFEAlchemicalNetworkPlanner.html#openfe.setup.RHFEAlchemicalNetworkPlanner 

### Building the Network Planner and getting an AlchemicalNetwork

Here we use the same ChemicalComponents, atom mapper, scorer, and network planner as used in the RBFE section above.

The `RHFEAlchemicalNetworkPlanner` will:

1. Create a new `LigandNetwork`.
2. Create two `Transformation` per `LigandNetwork` edge, one for the solvent and vacuum legs of the transformation, setting appropriate nonbonded methods and cutoffs for the periodic and nonperiodic simulations.
3. Combine all the `Transformation`s into an `AlchemicalNetwork`.

In [5]:
from openfe.setup.alchemical_network_planner import RHFEAlchemicalNetworkPlanner

# Build the planner
alchem_planner = RHFEAlchemicalNetworkPlanner(
    name="project", 
    mappers=[KartografAtomMapper()],
    mapping_scorer=default_lomap_score,
    ligand_network_planner=generate_minimal_spanning_network,
)

# Create the AlchemicalNetwork
alchemical_network = alchem_planner(
    ligands=ligands,
    solvent=solvent,
)